In [1]:
# Import Snowflake modules
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json

In [2]:
# Get account credentials from a json file
with open("data_scientist_auth.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    "role": "TASTY_BI",
    "warehouse": "TASTY_BI_WH",
    "database": "frostbyte_tasty_bytes",
    "schema": "analytics",
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

In [3]:
SENTIMENT_DATA = session.table("frostbyte_tasty_bytes_setup_s.raw_survey.sentiment_survey_ds")

TRUCK_DATA = session.table("frostbyte_tasty_bytes_setup_s.raw_pos.truck")

In [4]:
SENTIMENT_DATA.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SURVEY_ID"       |"SURVEY_TS"  |"TRUCK_ID"  |"COMMENTS"                                    |"NPS_SCORE"  |"NPS_CATEGORY"  |"OVERALL_SENTIMENT"  |"ASPECT_SENTIMENT"                           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|60509-SL          |2023-01-25   |291         |                                              |60.0         |Passive         |0.0                  |[                                            |
|                  |             |            |                                              |             |                |                     |  {                                          |
|                  |          

In [41]:
TRUCK_DATA.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"MENU_TYPE_ID"  |"PRIMARY_CITY"  |"REGION"    |"ISO_REGION"  |"COUNTRY"      |"ISO_COUNTRY_CODE"  |"FRANCHISE_FLAG"  |"YEAR"  |"MAKE"        |"MODEL"           |"EV_FLAG"  |"FRANCHISE_ID"  |"TRUCK_OPENING_DATE"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|3           |3               |San Mateo       |California  |CA            |United States  |US                  |1                 |2004    |Freightliner  |MT45 Utilimaster  |0          |2               |2021-10-01            |
|4           |4               |San Mateo       |California  |CA            |United State

In [43]:
SENTIMENT = TRUCK_DATA.join(SENTIMENT_DATA, SENTIMENT_DATA["TRUCK_ID"] == TRUCK_DATA["TRUCK_ID"], "inner")

In [44]:
SENTIMENT2 = SENTIMENT.dropna()

In [45]:
AVG_NPS_SCORE = SENTIMENT2.with_column("SURVEY_TS_YEAR", F.year("SURVEY_TS"))
AVG_NPS_SCORE = AVG_NPS_SCORE.groupBy(F.col("SURVEY_TS_YEAR")).agg(F.avg("NPS_SCORE"))
AVG_NPS_SCORE.show()

----------------------------------------
|"SURVEY_TS_YEAR"  |"AVG(NPS_SCORE)"   |
----------------------------------------
|2023              |70.0               |
|2019              |89.78290366350068  |
|2020              |82.65104808877929  |
|2021              |70.78996630475477  |
|2022              |73.91523935002196  |
----------------------------------------



In [51]:
NPS_COMPARE = session.table(name = "NPS_COMPARE")

In [52]:
NPS_COMPARE.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"CITY"  |"DATE"      |"ORDER_TOTAL"  |"TOTAL_QUANTITY"  |"DAYS_TO_NEXT_ORDER"  |"DAYS_SINCE_PRIOR_ORDER"  |"AVG_DAYS_BETWEEN_PURCHASES"  |"MAX_DAYS_WITHOUT_ITEM"  |"MIN_DAYS_WITHOUT_ITEM"  |"FREQUENCY"  |"MONETARYVALUE"  |"AGE"  |"NextPurchaseDayRange"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|81445          |Sydney  |2020-04-20  |104.0          |12                |29                    |64                        |64                            |64                       |

In [53]:
NPS_COMPARE = NPS_COMPARE.with_column("YEAR", F.year("DATE"))
NPS_COMPARE.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"CITY"  |"DATE"      |"ORDER_TOTAL"  |"TOTAL_QUANTITY"  |"DAYS_TO_NEXT_ORDER"  |"DAYS_SINCE_PRIOR_ORDER"  |"AVG_DAYS_BETWEEN_PURCHASES"  |"MAX_DAYS_WITHOUT_ITEM"  |"MIN_DAYS_WITHOUT_ITEM"  |"FREQUENCY"  |"MONETARYVALUE"  |"AGE"  |"NextPurchaseDayRange"  |"YEAR"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|81445          |Sydney  |2020-04-20  |104.0          |12                |29                    |64                        |64                            

In [54]:
AVG_NPDR = NPS_COMPARE.groupBy(F.col("YEAR")).agg(F.avg('"NextPurchaseDayRange"'))
AVG_NPDR.show()
AVG_NPS_SCORE.show()

----------------------------------------
|"YEAR"  |"AVG(NEXTPURCHASEDAYRANGE)"  |
----------------------------------------
|2020    |0.737171                     |
|2021    |0.634631                     |
|2022    |0.522950                     |
----------------------------------------

----------------------------------------
|"SURVEY_TS_YEAR"  |"AVG(NPS_SCORE)"   |
----------------------------------------
|2023              |70.0               |
|2019              |89.78290366350068  |
|2020              |82.65104808877929  |
|2021              |70.78996630475477  |
|2022              |73.91523935002196  |
----------------------------------------

